# Get network communities

This notebook gets network communities for the compendia (PAO1 and PA14) using different thresholds.

The output of this notebook are files for each threshold. These files have the following columns:
gene_id | group_id

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
import os
import pandas as pd
from core_acc_modules import paths
from rpy2.robjects import pandas2ri

pandas2ri.activate()

In [14]:
# Params
corr_threshold = 0.9

In [2]:
# Load expression data
pao1_compendium_filename = paths.PAO1_COMPENDIUM
pa14_compendium_filename = paths.PA14_COMPENDIUM

In [3]:
pao1_compendium = pd.read_csv(pao1_compendium_filename, sep="\t", header=0, index_col=0)
pa14_compendium = pd.read_csv(pa14_compendium_filename, sep="\t", header=0, index_col=0)

In [4]:
print(pao1_compendium.shape)
pao1_compendium.head()

(847, 5563)


,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA1905,PA0195,PA4812,PA0195.1,PA0457.1,PA1552.1,PA1555.1,PA3701,PA4724.1,PA5471.1
ERX541579,150.908951,63.968306,56.863002,86.013480,32.240954,48.322611,21.099413,21.219456,31.521424,12.168960,...,0.000000,16.606783,38.218939,8.611779,28.396413,110.943698,137.583940,101.058282,65.210599,0.000000
ERX541580,108.323515,56.079007,57.001674,73.421480,18.426169,46.572630,16.606703,18.595344,30.253393,15.936006,...,0.000000,13.904292,37.246074,11.777708,45.733000,94.799523,53.486191,82.753701,23.340646,38.113576
ERX541591,556.704228,182.309588,81.730476,157.348561,85.185006,116.643999,36.964524,31.973493,63.109930,16.140763,...,14.312595,73.032021,80.369628,49.312460,69.099677,61.622295,103.911157,186.030844,478.965167,0.000000
ERX541592,428.470250,178.510235,74.953217,152.142022,92.393069,95.317522,33.668009,31.262234,81.131515,20.126563,...,6.691671,114.618032,62.583672,56.426453,58.473904,55.311934,101.620394,207.409539,345.057960,0.000000
ERX676205,555.932721,530.701952,295.028891,336.887163,160.855884,196.799841,24.062303,127.448405,251.543954,29.933785,...,9.935461,59.660341,268.159400,40.929204,30.272774,284.146647,357.806954,260.651493,110.621722,0.000000


In [5]:
print(pa14_compendium.shape)
pa14_compendium.head()

(520, 5891)


,PA14_55610,PA14_55600,PA14_55590,PA14_55580,PA14_55570,PA14_55560,PA14_55550,PA14_55540,PA14_55530,PA14_55520,...,PA14_19205,PA14_17675,PA14_67975,PA14_36345,PA14_43405,PA14_38825,PA14_24245,PA14_28895,PA14_55117,PA14_59845
ERX1477379,11.030217,1.370300,0.683438,63.527772,34.586977,4.709054,5.274344,8.137393,5.310585,0.766974,...,10.688588,82.981151,63.650349,28.877599,7.956130,56.829502,269.183334,6.074311,29.079407,100.748813
ERX1477380,13.419626,3.039174,1.334504,105.003820,70.493150,8.849012,6.802855,18.784232,11.978251,2.088495,...,15.823345,91.090917,62.400165,24.819442,3.132517,31.325168,181.774134,10.866863,16.090103,233.317754
ERX1477381,13.155391,5.684725,7.208220,143.201487,107.849795,12.753058,8.868100,27.482294,13.951196,1.251785,...,28.406181,98.909427,71.429549,24.114657,6.515803,18.616579,184.357151,13.112427,13.970424,93.182566
ERX2174773,4.201447,4.194614,3.283683,3.273384,20.188057,3.804287,2.665245,2.650568,20.062716,3.755125,...,41.954387,35.821193,114.727681,104.737209,0.000000,391.722223,160.215744,18.980486,20.985404,182.605858
ERX2174774,5.016892,3.306859,3.446703,2.676762,17.199329,3.748607,1.618736,4.083091,21.225030,2.373772,...,44.138587,26.599121,105.848301,104.402335,0.000000,444.485136,168.538985,24.906987,19.775702,206.865218


## Make adjacency matrix

* Start with similarity matrix, S = [s_ij] = |cor(i,j)| \in [0,1] which represents the concordance between gene expression profiles for gene i and j.
* The adjacency matrix, A = [a_ij]  encodes the strength of the connection between gene i and j.
* For 'hard thresholding' a_ij = 1 if s_ij >= threshold, else 0.
* For 'soft thresholding' a_ij = |s_ij|^\beta . --
    * Beta is chosen based on 2 criteria.
    * 1) To get a network follows a scale free topology
    * Scale-free topology is defined by a network where the probability that a node is connected with k other node (the degree distribution p(k) of a network) decays as a power law p(k) ~ k^(-x)
    * This is what the “Weighted” in the name refers to -- using a weighted adjacency matrix, where the co-expression similarity is raised to a power
    * 2) Maintain a high number of connections
* Here we’re using a hard threshold cutoff
* We can select different correlation function, default is Pearson.

In [9]:
# Get perason correlation
pao1_corr = pao1_compendium.corr()
pa14_corr = pa14_compendium.corr()

In [12]:
pao1_corr.head()

,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA1905,PA0195,PA4812,PA0195.1,PA0457.1,PA1552.1,PA1555.1,PA3701,PA4724.1,PA5471.1
PA0001,1.000000,0.463956,0.569484,0.351438,0.245083,0.082922,-0.237419,0.327681,0.611781,-0.053683,...,-0.143054,0.067547,-0.046884,0.115943,0.095939,0.137961,0.123254,0.413492,-0.017787,-0.044556
PA0002,0.463956,1.000000,0.252376,0.748150,0.165336,-0.064989,-0.223772,0.474954,0.404363,-0.091208,...,-0.049508,-0.029025,0.068886,0.066085,-0.081798,0.364207,0.230212,0.295833,-0.003284,0.026462
PA0003,0.569484,0.252376,1.000000,0.370005,0.231063,0.305920,-0.227813,0.226757,0.414430,0.044911,...,-0.161805,-0.105521,0.058363,-0.032366,0.118532,0.204444,0.136097,0.330368,-0.000470,-0.053666
PA0004,0.351438,0.748150,0.370005,1.000000,0.178458,-0.013829,-0.289639,0.519269,0.406310,-0.039080,...,-0.141347,-0.146768,0.188109,-0.079627,0.004548,0.385017,0.265849,0.248906,-0.068307,-0.074792
PA0005,0.245083,0.165336,0.231063,0.178458,1.000000,0.494013,0.054159,0.548067,0.458342,0.600835,...,0.028263,0.156724,0.147440,0.288880,0.274439,0.158833,-0.076657,0.376051,0.323167,0.055236


In [18]:
# Create adjacency matrix using threshold defined above
pao1_adj = (pao1_corr >= corr_threshold).astype(int)
pa14_adj = (pa14_corr >= corr_threshold).astype(int)

In [19]:
pao1_adj.head()

,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA1905,PA0195,PA4812,PA0195.1,PA0457.1,PA1552.1,PA1555.1,PA3701,PA4724.1,PA5471.1
PA0001,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA0002,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA0003,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA0004,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PA0005,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Module detection
Detect modules. Get membership of genes that are closely related based on adjacency matrix (Function: `TOMsimilarity`)
* First we need to calculate the topological overlap measure (TOM)
* The topological overlap of two nodes reflects their similarity in terms of the commonality of the nodes they connect to
* We can group genes into modules based on their shared connections using clustering -- we’ll probably start with hierarchical clustering

In [6]:
%%R
library("WGCNA")

R[write to console]: Loading required package: dynamicTreeCut

R[write to console]: Loading required package: fastcluster

R[write to console]: 
Attaching package: ‘fastcluster’


R[write to console]: The following object is masked from ‘package:stats’:

    hclust


R[write to console]: 

R[write to console]: 
Attaching package: ‘WGCNA’


R[write to console]: The following object is masked from ‘package:stats’:

    cor




In [22]:
%%R -i pao1_adj -o tom_similarity

print(head(pao1_adj))
tom_similarity = TOMsimilarity(pao1_adj)

R[write to console]: Error in checkAdjMat(adjMat, min = if (TOMTypeC == 2) -1 else 0, max = 1) : 
  adjacency is not numeric
Calls: <Anonymous> ... <Anonymous> -> withVisible -> TOMsimilarity -> checkAdjMat



       PA0001 PA0002 PA0003 PA0004 PA0005 PA0006 PA0007 PA0008 PA0009 PA0010
PA0001      1      0      0      0      0      0      0      0      0      0
PA0002      0      1      0      0      0      0      0      0      0      0
PA0003      0      0      1      0      0      0      0      0      0      0
PA0004      0      0      0      1      0      0      0      0      0      0
PA0005      0      0      0      0      1      0      0      0      0      0
PA0006      0      0      0      0      0      1      0      0      0      0
       PA0011 PA0012 PA0013 PA0014 PA0015 PA0016 PA0017 PA0018 PA0019 PA0020
PA0001      0      0      0      0      0      0      0      0      0      0
PA0002      0      0      0      0      0      0      0      0      0      0
PA0003      0      0      0      0      0      0      0      0      0      0
PA0004      0      0      0      0      0      0      0      0      0      0
PA0005      0      0      0      0      0      0      0      0      0      0

In [ ]:
# TO DO
# 1-TOMsim?
# hclust(1-TOMsim)?
# cutTreeDynamic to get modules from htree?
# Get community membership for a single threshold
# Format and save output to have columns: gene_id | group_id
# Organize script to be able to run for multiple different thresholds as a script and save membership